In [359]:
#Import libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#!pip install missingno
import missingno as msno
# Set the option to display all rows
#pd.set_option('display.max_rows', None)
import re

In [360]:
df = pd.read_csv(r'C:\Users\dhami\Downloads\GitHub_Capstone_Project\CTREA-Dynamics\data\Real_Estate_Sales_2001-2020_GL.csv')
df.shape

C:\Users\dhami\AppData\Local\Temp\ipykernel_12608\1589272176.py:1: DtypeWarning: Columns (8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'C:\Users\dhami\Downloads\GitHub_Capstone_Project\CTREA-Dynamics\data\Real_Estate_Sales_2001-2020_GL.csv')


(997213, 14)

In [361]:
df.drop('Serial Number', axis=1, inplace=True)

In [362]:
#based on data assessment, we found 10 percent may losss due to IQR method
def detect_outliers(data):
    numeric_data = data.select_dtypes(exclude='object')
    columns = list(numeric_data.columns)
    outliers_info = []

    for column in columns:
        # IQR method
        q1 = numeric_data[column].quantile(0.25)
        q3 = numeric_data[column].quantile(0.75)
        iqr = q3 - q1
        lower_limit = q1 - 1.5 * iqr
        upper_limit = q3 + 1.5 * iqr

        # Detect outliers
        outliers = (numeric_data[column] < lower_limit) | (numeric_data[column] > upper_limit)

        # Count the number of outliers
        num_outliers = outliers.sum()
        percentage_outliers = round((outliers.sum()/numeric_data.shape[0]) * 100,2)

        # Append information to the list
        outliers_info.append({
            'Column': column,
            'Lower Limit': lower_limit,
            'Upper Limit': upper_limit,
            'Num Outliers': num_outliers,
            'Outlier Indices': numeric_data[outliers].index.tolist(),
            'Outliers_percentage' : percentage_outliers
        })

    # Convert the list of dictionaries to a DataFrame
    outliers_df = pd.DataFrame(outliers_info)

    return outliers_df

# Example usage
outliers_df = detect_outliers(df)
outliers_df


,Column,Lower Limit,Upper Limit,Num Outliers,Outlier Indices,Outliers_percentage
0,List Year,1986.000000,2034.00000,0,[],0.00
1,Assessed Value,-119340.000000,432500.00000,90623,"[3, 4, 5, 8, 12, 13, 35, 40, 55, 74, 82, 84, 8...",9.09
2,Sale Amount,-197500.000000,702500.00000,82900,"[3, 4, 5, 8, 33, 35, 40, 44, 55, 74, 82, 84, 8...",8.31
3,Sales Ratio,0.038836,1.23314,100129,"[6, 32, 50, 60, 75, 86, 94, 96, 127, 133, 139,...",10.04


In [363]:
lower_limit = outliers_df.iloc[3,1]
upper_limit = outliers_df.iloc[3,2]
(lower_limit,upper_limit)

(0.03883588900000007, 1.2331401849999999)

In [364]:
data = df[(df['Sales Ratio'] > lower_limit) & (df['Sales Ratio'] < upper_limit)]
data.head()

,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location
0,2020,09/13/2021,Ansonia,230 WAKELEE AVE,150500.0,325000.0,0.4630,Commercial,NaN,NaN,NaN,NaN,NaN
1,2020,10/02/2020,Ashford,390 TURNPIKE RD,253000.0,430000.0,0.5883,Residential,Single Family,NaN,NaN,NaN,NaN
2,2020,03/09/2021,Avon,5 CHESTNUT DRIVE,130400.0,179900.0,0.7248,Residential,Condo,NaN,NaN,NaN,NaN
3,2020,04/13/2021,Avon,111 NORTHINGTON DRIVE,619290.0,890000.0,0.6958,Residential,Single Family,NaN,NaN,NaN,NaN
4,2020,07/02/2021,Avon,70 FAR HILLS DRIVE,862330.0,1447500.0,0.5957,Residential,Single Family,NaN,NaN,NaN,NaN


In [365]:
data.shape

(897084, 13)

In [366]:
round(data.describe(include='all'),2).T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
List Year,897084.0,NaN,NaN,NaN,2010.15,6.32,2001.0,2004.0,2010.0,2016.0,2020.0
Date Recorded,897084,6313,07/01/2005,840,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Town,897084,170,Stamford,30139,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Address,897070,665089,MULTI ADDRESSES,444,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Assessed Value,897084.0,NaN,NaN,NaN,234248.08,751951.6,70.0,89510.0,140410.0,226900.0,142858700.0
Sale Amount,897084.0,NaN,NaN,NaN,402345.88,1610323.16,300.0,158000.0,240000.0,380000.0,395500000.0
Sales Ratio,897084.0,NaN,NaN,NaN,0.62,0.21,0.04,0.48,0.61,0.73,1.23
Property Type,554579,11,Single Family,364047,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Residential Type,549989,5,Single Family,406206,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Non Use Code,193673,103,25 - Other,42263,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [367]:
outliers_df_1 = detect_outliers(data)
outliers_df_1

,Column,Lower Limit,Upper Limit,Num Outliers,Outlier Indices,Outliers_percentage
0,List Year,1986.000000,2034.00000,0,[],0.00
1,Assessed Value,-116575.000000,432985.00000,77611,"[3, 4, 5, 8, 12, 13, 35, 40, 55, 74, 82, 84, 8...",8.65
2,Sale Amount,-175000.000000,713000.00000,78242,"[3, 4, 5, 8, 33, 35, 40, 44, 55, 74, 82, 84, 8...",8.72
3,Sales Ratio,0.100224,1.11296,25770,"[69, 283, 370, 379, 383, 574, 628, 654, 701, 7...",2.87


In [368]:
data.shape

(897084, 13)

In [369]:
data['Non Use Code'].unique()

array([nan, '14 - Foreclosure', '25 - Other', '01 - Family',
       '12 - Non Buildable Lot', '7', '24', '07 - Change in Property',
       '1', '17', '6', '24 - Plottage', '28 - Use Assessment', '09 - Tax',
       '18 - In Lieu Of Foreclosure', '17 - Two Towns', '10 - A Will',
       '06 - Portion of Property', '08 - Part Interest',
       '15 - Government Agency', '16 - Charitable Group',
       '11 - Court Order', '26 - Rehabilitation Deferred',
       '03 - Inter Corporation', '22 - Money and Personal Property',
       'Single Family', '04 - Correcting Deed', '05 - Deed Date',
       '27 - CRUMBLING FOUNDATION ASSESSMENT REDUCTION', '13 - Bankrupcy',
       '30 - Auction', '02 - Love and Affection',
       '21 - Personal Property Exchange', '23 - Zoning', '14', '25', '3',
       '20 - Cemetery', '8', '19 - Easement', '28', '10', '12', '15',
       '21', '16', '11', '18', '13', '4', '19', '22', '2', '23', '30',
       '9', '5', '26', '27', 7.0, 8.0, 13.0, 6.0, 25.0, 28.0, 10.0, 27.0,

In [370]:

# Split the 'Non Use Code' column
split_data = data['Non Use Code'].astype(str).str.split('-', n=1, expand=True)

# Create 'Number' and 'Word' columns based on the split
data['Number'] = np.where(split_data[1].isnull(), split_data[0], split_data[0])
data['Word'] = split_data[1]

# Display the modified DataFrame
#print(data.head())


C:\Users\dhami\AppData\Local\Temp\ipykernel_12608\3428283491.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Number'] = np.where(split_data[1].isnull(), split_data[0], split_data[0])
C:\Users\dhami\AppData\Local\Temp\ipykernel_12608\3428283491.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Word'] = split_data[1]


In [371]:
data[['Non Use Code', 'Number','Word']].sample(20)

,Non Use Code,Number,Word
63936,NaN,nan,None
347023,NaN,nan,None
346147,NaN,nan,None
254660,NaN,nan,None
356192,NaN,nan,None
588913,NaN,nan,None
664516,NaN,nan,None
237196,NaN,nan,None
422004,NaN,nan,None
704111,NaN,nan,None


In [372]:
data['Number'].unique()

array(['nan', '14 ', '25 ', '01 ', '12 ', '7', '24', '07 ', '1', '17',
       '6', '24 ', '28 ', '09 ', '18 ', '17 ', '10 ', '06 ', '08 ', '15 ',
       '16 ', '11 ', '26 ', '03 ', '22 ', 'Single Family', '04 ', '05 ',
       '27 ', '13 ', '30 ', '02 ', '21 ', '23 ', '14', '25', '3', '20 ',
       '8', '19 ', '28', '10', '12', '15', '21', '16', '11', '18', '13',
       '4', '19', '22', '2', '23', '30', '9', '5', '26', '27', '7.0',
       '8.0', '13.0', '6.0', '25.0', '28.0', '10.0', '27.0', '11.0',
       '14.0', '17.0', '1.0', '16.0', '3.0', '4.0', '24.0', '18.0', '2.0',
       '15.0', '29.0', '22.0', '30.0', '21.0', '5.0', '12.0', '23.0',
       '9.0', '19.0', '20.0', '26.0', '68.0', '31.0', '73.0', '38.0',
       '37.0', '55.0', '32.0', '29', '88', '37', '31', '34', '74', '32',
       '29 '], dtype=object)

In [373]:
data['Word'].unique()

array([None, ' Foreclosure', ' Other', ' Family', ' Non Buildable Lot',
       ' Change in Property', ' Plottage', ' Use Assessment', ' Tax',
       ' In Lieu Of Foreclosure', ' Two Towns', ' A Will',
       ' Portion of Property', ' Part Interest', ' Government Agency',
       ' Charitable Group', ' Court Order', ' Rehabilitation Deferred',
       ' Inter Corporation', ' Money and Personal Property',
       ' Correcting Deed', ' Deed Date',
       ' CRUMBLING FOUNDATION ASSESSMENT REDUCTION', ' Bankrupcy',
       ' Auction', ' Love and Affection', ' Personal Property Exchange',
       ' Zoning', ' Cemetery', ' Easement', ' No Consideration'],
      dtype=object)

In [374]:
data[data['Number'] == 'Single Family'].index

Index([1964, 932292, 977979], dtype='int64')

In [375]:
rows_to_drop = [1964, 932292, 977979]
data = data.drop(rows_to_drop)

In [376]:
data[data['Number'] == 'Single Family']

,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location,Number,Word


In [377]:
data.sample(4)

,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location,Number,Word
435176,2007,11/05/2007,Woodbury,266 WASHINGTON RD,200760.0,406000.0,0.494483,NaN,NaN,NaN,NaN,NaN,NaN,nan,None
294095,2004,06/07/2005,Danbury,12 CORNELL RD,81500.0,185000.0,0.440541,NaN,NaN,7.0,NaN,NaN,NaN,7.0,None
469947,2007,11/09/2007,Stratford,86 MELVILLE ST,137830.0,240000.0,0.574292,Single Family,Single Family,NaN,NaN,NaN,POINT (-73.13964 41.19392),nan,None
272359,2004,11/10/2004,Ledyard,64 IRON ST,107310.0,238000.0,0.450882,NaN,NaN,NaN,NaN,NaN,NaN,nan,None


In [378]:
data.to_csv('outliers_removed.csv')

In [379]:
data.drop(columns=['Non Use Code','Assessor Remarks','OPM remarks','Location','Number'], axis=1, inplace=True)

In [380]:
# Rename the 'Word' column to 'Non-Code'
data.rename(columns={'Word': 'Non-Code'}, inplace=True)

In [381]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 897081 entries, 0 to 997212
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   List Year         897081 non-null  int64  
 1   Date Recorded     897081 non-null  object 
 2   Town              897081 non-null  object 
 3   Address           897067 non-null  object 
 4   Assessed Value    897081 non-null  float64
 5   Sale Amount       897081 non-null  float64
 6   Sales Ratio       897081 non-null  float64
 7   Property Type     554579 non-null  object 
 8   Residential Type  549989 non-null  object 
 9   Non-Code          142410 non-null  object 
dtypes: float64(3), int64(1), object(6)
memory usage: 75.3+ MB


In [382]:
data['Property Type'].fillna('Miscellaneous', inplace=True)
data['Residential Type'].fillna('Others', inplace=True)
data['Non-Code'].fillna('Not defined', inplace=True)

In [383]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 897081 entries, 0 to 997212
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   List Year         897081 non-null  int64  
 1   Date Recorded     897081 non-null  object 
 2   Town              897081 non-null  object 
 3   Address           897067 non-null  object 
 4   Assessed Value    897081 non-null  float64
 5   Sale Amount       897081 non-null  float64
 6   Sales Ratio       897081 non-null  float64
 7   Property Type     897081 non-null  object 
 8   Residential Type  897081 non-null  object 
 9   Non-Code          897081 non-null  object 
dtypes: float64(3), int64(1), object(6)
memory usage: 75.3+ MB


In [384]:
data['Property Type'].unique()

array(['Commercial', 'Residential', 'Vacant Land', 'Miscellaneous',
       'Apartments', 'Industrial', 'Public Utility', 'Condo',
       'Two Family', 'Single Family', 'Four Family', 'Three Family'],
      dtype=object)

In [385]:
data['Residential Type'].unique()

array(['Others', 'Single Family', 'Condo', 'Two Family', 'Three Family',
       'Four Family'], dtype=object)

In [386]:
data['Non-Code'].unique()

array(['Not defined', ' Foreclosure', ' Other', ' Family',
       ' Non Buildable Lot', ' Change in Property', ' Plottage',
       ' Use Assessment', ' Tax', ' In Lieu Of Foreclosure', ' Two Towns',
       ' A Will', ' Portion of Property', ' Part Interest',
       ' Government Agency', ' Charitable Group', ' Court Order',
       ' Rehabilitation Deferred', ' Inter Corporation',
       ' Money and Personal Property', ' Correcting Deed', ' Deed Date',
       ' CRUMBLING FOUNDATION ASSESSMENT REDUCTION', ' Bankrupcy',
       ' Auction', ' Love and Affection', ' Personal Property Exchange',
       ' Zoning', ' Cemetery', ' Easement', ' No Consideration'],
      dtype=object)

In [387]:
# Convert 'Date Recorded' to datetime format
data['Date Recorded'] = pd.to_datetime(data['Date Recorded'], format='%m/%d/%Y')

# Create a new column 'List Year' by extracting the year from 'Date Recorded'
data['Date Recorded_Year'] = data['Date Recorded'].dt.year
data['Date Recorded_Month'] = data['Date Recorded'].dt.month
data['Date Recorded_Date'] = data['Date Recorded'].dt.day

# Drop the 'Year' column if you want to remove it
data.drop('Date Recorded', axis=1, inplace=True)


# Extract street numbers using a regular expression
data['Street Number'] = data['Address'].str.extract(r'(\d+)')
data.drop('Address', axis=1, inplace=True)


In [388]:
data.sample(10)

,List Year,Town,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non-Code,Date Recorded_Year,Date Recorded_Month,Date Recorded_Date,Street Number
968578,2019,Norwalk,946850.0,1550001.0,0.610900,Single Family,Single Family,Not defined,2019,12,6,7
117357,2001,Stamford,254380.0,619000.0,0.410953,Miscellaneous,Others,Not defined,2002,5,17,195
451621,2007,Groton,158340.0,224900.0,0.704046,Single Family,Single Family,Not defined,2008,7,29,5
825041,2017,Branford,173800.0,255000.0,0.681569,Single Family,Single Family,Not defined,2018,2,6,145
15988,2020,Killingly,104580.0,236000.0,0.443100,Residential,Single Family,Not defined,2021,5,25,28
451004,2006,Norwalk,91700.0,220000.0,0.416818,Condo,Condo,Not defined,2007,5,15,149
947863,2019,Bridgeport,142850.0,281520.0,0.507400,Two Family,Two Family,Not defined,2020,2,4,222
295332,2004,Danbury,160500.0,337000.0,0.476261,Miscellaneous,Others,Not defined,2005,7,21,59
398319,2006,Cromwell,153860.0,360000.0,0.427389,Condo,Condo,Not defined,2007,8,23,26
480901,2007,Vernon,113000.0,184000.0,0.614130,Single Family,Single Family,Not defined,2008,4,9,32


In [389]:
data.to_csv('model_building_dataset.csv')